In [4]:
from google.colab import auth
from google.auth import default
from google.auth.transport.requests import Request
import gspread
import pandas as pd
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, Border, Side
from openpyxl.utils import get_column_letter

# Đăng nhập và thiết lập kết nối
auth.authenticate_user()
creds, _ = default()
creds.refresh(Request())
gc = gspread.authorize(creds)
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1JG0DlXMc_iw3G-5N2_IYzloQWAVY3Jj_Pn30G4rW3JQ/edit?usp=sharing')

# Lấy dữ liệu và tạo DataFrame
worksheet = sheet.get_worksheet(0)
all_data = worksheet.get_all_values()
df = pd.DataFrame(all_data[1:], columns=all_data[0])

# Làm sạch và chọn dữ liệu cần thiết
df.columns = df.columns.str.strip()
report_df = df[['Dấu thời gian', 'Nhập mã học sinh/sinh viên',
                'Phản ánh của học sinh/sinh viên về tình hình ở lại trưa (nếu có)', 'Họ và tên HSSV']].copy()

# Thêm cột STT
report_df.reset_index(drop=True, inplace=True)
report_df.index += 1
report_df.insert(0, 'STT', report_df.index)

# Tạo workbook và thiết lập định dạng
wb = Workbook()
ws = wb.active
ws.title = "Báo cáo HSSV ở lại trưa"

# Định dạng tiêu đề
ws.merge_cells('A1:E1')
ws['A1'] = 'BÁO CÁO TÌNH HÌNH HSSV Ở LẠI TRƯA HÀNG NGÀY'
ws['A1'].font = Font(size=14, bold=True)
ws['A1'].alignment = Alignment(horizontal='center')

# Thêm dữ liệu và định dạng cột
thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
for r_idx, row in enumerate(dataframe_to_rows(report_df, index=False, header=True), start=2):
    for c_idx, value in enumerate(row, start=1):
        cell = ws.cell(row=r_idx, column=c_idx, value=value)
        cell.border = thin_border
        cell.alignment = Alignment(horizontal='center')
        if r_idx == 2:
            cell.font = Font(bold=True)

# Điều chỉnh độ rộng cột dựa trên nội dung của các cột
for col in ws.columns:
    max_length = 0
    column_letter = get_column_letter(col[0].column)  # Lấy chữ cái của cột từ thuộc tính column
    if column_letter == 'A':  # Giả sử cột 'STT' là cột đầu tiên
        ws.column_dimensions[column_letter].width = 10  # Đặt độ rộng cố định cho cột 'STT'
    else:
        for cell in col:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        adjusted_width = (max_length + 2) * 1.2  # Thêm một chút không gian thêm
        ws.column_dimensions[column_letter].width = adjusted_width

# Thêm ghi chú cuối bảng
last_row = ws.max_row + 1
ws.merge_cells(f'A{last_row}:E{last_row}')
ws[f'A{last_row}'] = "PHÒNG THANH TRA, KHẢO THÍ - CÔNG TÁC HSSV"
ws[f'A{last_row}'].font = Font(bold=True)
ws[f'A{last_row}'].alignment = Alignment(horizontal='right')

# Lưu workbook
file_name = f'Báo cáo tình hình HSSV ở lại trưa ngày_{datetime.today().strftime("%d-%m-%Y")}.xlsx'
wb.save(file_name)

print(f"Báo cáo đã được lưu thành file Excel: {file_name}")


Báo cáo đã được lưu thành file Excel: Báo cáo ngày_19-09-2024.xlsx
